#Modeling U.S Gun-Homcide Rates using SVI
###A comprehensive analysis by Adam Sam

<img src="https://www.bridgemi.com/sites/default/files/styles/full_width_image/public/2022-05/gun%20shutterstock.jpg?itok=AUziubj7" width="520" height="300">



##Datasets
Gun homicide: https://wonder.cdc.gov/controller/saved/D158/D429F689

SVI: https://www.atsdr.cdc.gov/place-health/php/svi/index.html

**-Are there correlations between features?**

The dataset includes multiple features grouped into four themes: Socioeconomic Status, Household Characteristics, Racial & Ethnic Minority Status, and Housing Type & Transportation.

Features like poverty (EP_POV150),unemployment (EP_UNEMP), and lack of high school diploma (EP_NOHSDP) could likely be positively correlated.

Features such as elderly population (EP_AGE65) and disability (EP_DISABL) may also be correlated.

Variables starting with "RPL_" represent the final overall social vulnerability rankings calculated based on the preceding determining variables (non-RPL variables). These are likely correlated with said variables, and will be separated.

A correlation matrix could quantitatively confirm these relationships.

**-Are there outliers?**
Washington D.C may be an outlier, as it has the highest homicide rate of 21 per 100k.

Some other interesting outliers:
EP_MINRTY (percentage who are considered minorities): Hawaii (73.76) and D.C (63.7) are much higher than the average.

EP_UNINSUR (percentage who are uninsured): Texas (17.41) and Oklahoma (15.16) are outliers compared to states like Massachusetts (3.01).

**-Will all features be used? Why or why not?**

Not all features in the dataset were used, as some were deemed as adjunct, or irrelevant for measuring social vulnerability; particularly measures of ethnicity composition.

The following variables are labeled as adjunct:

-estimate of daytime population derived from LandScan 2021 estimates

-ACS estimates for households without an internet subscription

-ACS estimates for Hispanic/Latino persons, Not Hispanic or Latino Black/African American persons, Not Hispanic or Latino Asian persons, Not
Hispanic or Latino American Indian and Alaska Native persons, Not Hispanic or Latino Native Hawaiian and Other Pacific Islander persons, Not Hispanic
or Latino persons of two or more races, and Not Hispanic or Latino persons of some other race

Variables that are not percentages were not used (raw values). Only the variables starting with "EP_" were used, as they represent percentages.

Variables starting with "RPL_" represent the final overall social vulnerability rankings calculated based on the preceding determining variables (non-RPL variables). These are likely correlated with said varaibles, and will be separated.

The RPL_THEMES feature is of main interest, as it is a ranking between 0 and 1 measuring social vulnerability considering all themes (socialeconomic, household, racial, transportation).

RPL_THEME1 to RPL_THEME4 are separate rankings for individual layers of the SVI hierarchy.

**-Is the dataset balanced?**
The dataset appears balanced in terms of geographic representation, as it includes all U.S. states and counties with consistent metrics. However, balance depends on the context:

However, the homicide rate per 100k varies widely (example: 1.28 in New Hampshire vs. 20.99 in D.C.), suggesting potential skewness. This could indicate imbalance if modeling homicide rates.

**-What feature engineering is needed?**

-Manually calculating homicide rate for rows where it is "Unreliable."

-Scaling the data using min=max scaling.

-Dimensionality reduction using PCA

**-Preprocessing steps**

-Grouping the data by states

-Replacing "Unreliable" values within death rate column with the proper values by manually calculating Deaths / Population * 100k.

-Irrelevant columns were removed, including:

*   Flags representing states fell within a certain category (0 or 1)
*   Extraneous state codes and abbreviations
*   Exact margin of error values
*   Miscellaneous information not used in the measurement of SVI (ethnicity composition)

##DATASET LOADING

In [322]:
""" ADAM SAM
steps to connect to lambda machine for co-lab
1. ssh -L 8888:localhost:8888 sama1@lab.cs.wit.edu -p 50004

2. jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com'
--port=8888 --NotebookApp.port_retries=0

3.
On your PC, go to Google Colab and select "Connect to a local runtime" (top
right corner).

In the URL box, enter the URL with the token you received from the output of
the Jupyter Notebook command in Step 2.
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%load_ext memory_profiler

####MAKE SURE TO CD TO ML_PROJECT DIR FIRST!#####
#!cd ML_PROJECT
#scp -P 50004 "C:\Users\sama1\Downloads\f2\svi\SVI_FINAL_CLEAN_2022.csv" sama1@lab.cs.wit.edu:"~/ML_PROJECT"
#loading dataset
data = pd.read_csv("SVI_FINAL_CLEAN_2022.csv")
#data.head()

#drop "Deaths" and "Population" and "STATE"
data = data.drop(["STATE","Deaths", "Population"], axis=1)
display(data.head())

target = data.iloc[:,-1]

#display(target)

data = data.drop(columns=data.columns[-1])
display(data.head())
#separate "RPL_" columns
data_with_ranks = data.loc[:, data.columns.str.startswith('RPL_')]
data = data.loc[:, ~data.columns.str.startswith('RPL_')]

display(data_with_ranks.head())
display(data.head())


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


,EP_POV150,EP_UNEMP,EP_HBURD,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,...,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES,Homicide Rate Per 100k
0,29.850746,5.905970,22.014925,15.489552,9.804478,19.061194,21.622388,19.011940,6.911940,0.822388,...,21.141791,1.613433,6.574627,2.894030,0.693924,0.625413,0.674607,0.531418,0.671582,12.297272
1,20.593333,7.893333,17.406667,8.780000,15.680000,14.456667,23.646667,14.080000,6.016667,1.970000,...,5.250000,10.753333,21.553333,9.890000,0.478500,0.358120,0.832597,0.805073,0.604770,5.589006
2,28.013333,7.146667,22.926667,13.900000,11.333333,21.880000,22.106667,16.086667,6.273333,4.686667,...,20.173333,5.913333,5.826667,3.180000,0.749767,0.752953,0.839607,0.787700,0.829213,6.658335
3,31.020000,5.973333,22.014667,13.812000,8.330667,19.936000,22.144000,20.978667,6.780000,1.081333,...,16.174667,2.529333,6.849333,3.348000,0.673488,0.725617,0.560131,0.593488,0.690255,9.521818
4,21.489655,6.765517,28.112069,14.094828,6.887931,18.608621,21.793103,13.675862,5.603448,6.181034,...,7.115517,5.760345,5.698276,3.327586,0.631514,0.551059,0.802922,0.709186,0.690778,4.294205


,EP_POV150,EP_UNEMP,EP_HBURD,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,...,EP_MUNIT,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES
0,29.850746,5.905970,22.014925,15.489552,9.804478,19.061194,21.622388,19.011940,6.911940,0.822388,...,3.277612,21.141791,1.613433,6.574627,2.894030,0.693924,0.625413,0.674607,0.531418,0.671582
1,20.593333,7.893333,17.406667,8.780000,15.680000,14.456667,23.646667,14.080000,6.016667,1.970000,...,5.036667,5.250000,10.753333,21.553333,9.890000,0.478500,0.358120,0.832597,0.805073,0.604770
2,28.013333,7.146667,22.926667,13.900000,11.333333,21.880000,22.106667,16.086667,6.273333,4.686667,...,4.800000,20.173333,5.913333,5.826667,3.180000,0.749767,0.752953,0.839607,0.787700,0.829213
3,31.020000,5.973333,22.014667,13.812000,8.330667,19.936000,22.144000,20.978667,6.780000,1.081333,...,3.037333,16.174667,2.529333,6.849333,3.348000,0.673488,0.725617,0.560131,0.593488,0.690255
4,21.489655,6.765517,28.112069,14.094828,6.887931,18.608621,21.793103,13.675862,5.603448,6.181034,...,9.300000,7.115517,5.760345,5.698276,3.327586,0.631514,0.551059,0.802922,0.709186,0.690778


,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES
0,0.693924,0.625413,0.674607,0.531418,0.671582
1,0.478500,0.358120,0.832597,0.805073,0.604770
2,0.749767,0.752953,0.839607,0.787700,0.829213
3,0.673488,0.725617,0.560131,0.593488,0.690255
4,0.631514,0.551059,0.802922,0.709186,0.690778


,EP_POV150,EP_UNEMP,EP_HBURD,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,EP_MINRTY,EP_MUNIT,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ
0,29.850746,5.905970,22.014925,15.489552,9.804478,19.061194,21.622388,19.011940,6.911940,0.822388,36.023881,3.277612,21.141791,1.613433,6.574627,2.894030
1,20.593333,7.893333,17.406667,8.780000,15.680000,14.456667,23.646667,14.080000,6.016667,1.970000,54.526667,5.036667,5.250000,10.753333,21.553333,9.890000
2,28.013333,7.146667,22.926667,13.900000,11.333333,21.880000,22.106667,16.086667,6.273333,4.686667,50.940000,4.800000,20.173333,5.913333,5.826667,3.180000
3,31.020000,5.973333,22.014667,13.812000,8.330667,19.936000,22.144000,20.978667,6.780000,1.081333,26.834667,3.037333,16.174667,2.529333,6.849333,3.348000
4,21.489655,6.765517,28.112069,14.094828,6.887931,18.608621,21.793103,13.675862,5.603448,6.181034,48.439655,9.300000,7.115517,5.760345,5.698276,3.327586


#MODEL SELECTION BASELINE

In [323]:
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import time,sys
from io import StringIO

#BASELINE training on unscaled data!
#split data
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)


#linear regression. NOTE: Linear regression has no hyperparameters, so no CV is needed
lin_reg = LinearRegression()

t0 = time.time()
%memit lin_reg.fit(X_train, y_train)
t1 = time.time()
base_lin_train_time = t1 - t0

t0 = time.time()
base_y_pred_lin = lin_reg.predict(X_test)
t1 = time.time()
base_lin_infer_time = t1 - t0
#SVR
#Cross-validate SVR with GRIDSEARCH
param_grid = {'C': np.linspace(0.1, 1, 7), #higher C means less tolerance, lower means more tolerance
              'kernel': ['linear', 'rbf', 'poly'],
              'degree': [2, 3, 4,5],
              'epsilon':np.linspace(0.01, 0.1, 10)}





t0 = time.time()
grid_search = GridSearchCV(SVR(), param_grid, n_jobs=-1,cv=5, verbose=True, scoring = "neg_root_mean_squared_error")
grid_search.fit(X_train, y_train)
t1 = time.time()
base_svr_train_time = t1 - t0

svr = grid_search.best_estimator_

t0 = time.time()
base_y_pred_svr = svr.predict(X_test)
t1 = time.time()
base_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost
"""
n_estimators: The number of trees in the ensemble, often increased until no further improvements are seen.
max_depth: The maximum depth of each tree, often values are between 1 and 10.
eta: The learning rate used to weight each model, often set to small values such as 0.3, 0.1, 0.01, or smaller.
subsample: The number of samples (rows) used in each tree, set to a value between 0 and 1, often 1.0 to use all samples.
colsample_bytree: Number of features (columns) used in each tree, set to a value between 0 and 1, often 1.0 to use all features.
"""
param_grid = {'n_estimators': [300],
              'n_jobs':[-1],
              'gamma': [1,1.5],
              'max_depth': [5,8],
              'eta': [0.1,0.3],
              'subsample':[1],
              'colsample_bytree':[0.75],
              'reg_lambda':[0.5,1],
              'reg_alpha':[3,2.5],
              }

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, n_jobs=-1,scoring = "neg_root_mean_squared_error", verbose=True)
t0 = time.time()
grid_search = grid_search.fit(X_train, y_train)
t1 = time.time()
base_xgb_train_time = t1 - t0

xgb_reg = grid_search.best_estimator_

t0 = time.time()
base_y_pred_xgb = xgb_reg.predict(X_test)
t1= time.time()
base_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)



peak memory: 605.98 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 840 candidates, totalling 4200 fits
Best SVR parameters: {'C': 0.1, 'degree': 5, 'epsilon': 0.01, 'kernel': 'poly'}
Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}


In [324]:
#evaluate baseline regression performances
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
#lin reg
lin_reg_mse = mean_squared_error(y_test, base_y_pred_lin)
lin_reg_r2 = r2_score(y_test, base_y_pred_lin)

#svr
svr_mse = mean_squared_error(y_test, base_y_pred_svr)
svr_r2 = r2_score(y_test, base_y_pred_svr)

#xgb
xgb_mse = mean_squared_error(y_test, base_y_pred_xgb)
xgb_r2 = r2_score(y_test, base_y_pred_xgb)

#print errors
print(f"Linear Regression MSE: {lin_reg_mse}")
print(f"Linear Regression R^2: {lin_reg_r2}")
print(f"SVR MSE: {svr_mse}")
print(f"SVR R^2: {svr_r2}")
print(f"XGBoost MSE: {xgb_mse}")
print(f"XGBoost R^2: {xgb_r2}")

#training times
print(f"Linear Regression Training Time: {base_lin_train_time} seconds")
print(f"Base SVR Training Time: {base_svr_train_time} seconds")
print(f"Base XGBoost Training Time: {base_xgb_train_time} seconds")


Linear Regression MSE: 6.021257024000639
Linear Regression R^2: 0.6646905299320609
SVR MSE: 6.761787652845931
SVR R^2: 0.6234521420443748
XGBoost MSE: 5.910049667541645
XGBoost R^2: 0.6708834028842183
Linear Regression Training Time: 0.6738419532775879 seconds
Base SVR Training Time: 2.3886160850524902 seconds
Base XGBoost Training Time: 2.0220565795898438 seconds


##EXPERIMENT 1: FEATURE SCALING

In [325]:
#normalize using minmax()
#standard scaler
from sklearn.preprocessing import StandardScaler

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

#SCALE ONLY THE FEATURES!
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


#TRAINING ON SCALED DATA!
#linear regression. NOTE: Linear regression has no hyperparameters, so no CV is needed
lin_reg = LinearRegression()

t0 = time.time()
%memit lin_reg.fit(X_train, y_train)
t1 = time.time()
scale_lin_train_time = t1 - t0

t0 = time.time()
scale_y_pred_lin = lin_reg.predict(X_test)
t1 = time.time()
scale_lin_infer_time = t1 - t0

#SVR
param_grid = {'C': np.linspace(0.1, 1, 7), #higher C means less tolerance, lower means more tolerance
              'kernel': ['linear', 'rbf', 'poly'],
              'degree': [2, 3, 4,5],
              'epsilon':np.linspace(0.01, 0.1, 10)}

grid_search = GridSearchCV(SVR(), param_grid, cv=5, n_jobs=-1, verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
scale_svr_train_time = t1 - t0

t0 = time.time()
scale_y_pred_svr = grid_search.best_estimator_.predict(X_test)
t1 = time.time()
scale_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost

param_grid = {'n_estimators': [300],
              'n_jobs':[-1],
              'gamma': [1,1.5],
              'max_depth': [5,8],
              'eta': [0.1,0.3],
              'subsample':[1],
              'colsample_bytree':[0.75],
              'reg_lambda':[0.5,1],
              'reg_alpha':[3,2.5],
              }
#Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, n_jobs=-1,cv=5, scoring = "neg_root_mean_squared_error", verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
scale_xgb_train_time = t1 - t0

t0 = time.time()
scale_y_pred_xgb = grid_search.best_estimator_.predict(X_test)
t1 = time.time()
scale_xgb_infer_time = t1 - t0



print(f"Best XGBoost parameters:%s" % grid_search.best_params_)

peak memory: 605.98 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 840 candidates, totalling 4200 fits
peak memory: 605.98 MiB, increment: 0.00 MiB
Best SVR parameters: {'C': 1.0, 'degree': 2, 'epsilon': 0.01, 'kernel': 'linear'}
Fitting 5 folds for each of 32 candidates, totalling 160 fits
peak memory: 605.98 MiB, increment: 0.00 MiB
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}


In [326]:
#evaluate baseline regression performances
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
#lin reg
scale_lin_reg_mse = mean_squared_error(y_test, scale_y_pred_lin)
scale_lin_reg_r2 = r2_score(y_test, scale_y_pred_lin)

#svr
scale_svr_mse = mean_squared_error(y_test, scale_y_pred_svr)
scale_svr_r2 = r2_score(y_test, scale_y_pred_svr)

#xgb
scale_xgb_mse = mean_squared_error(y_test, scale_y_pred_xgb)
scale_xgb_r2 = r2_score(y_test, scale_y_pred_xgb)

#print errors
print(f"Linear Regression MSE: {scale_lin_reg_mse}")
print(f"Linear Regression R^2: {scale_lin_reg_r2}")
print(f"SVR MSE: {scale_svr_mse}")
print(f"SVR R^2: {scale_svr_r2}")
print(f"XGBoost MSE: {scale_xgb_mse}")
print(f"XGBoost R^2: {scale_xgb_r2}")

#training times
print(f"Scaled Linear Regression Training Time: {scale_lin_train_time} seconds")
print(f"Scaled SVR Training Time: {scale_svr_train_time} seconds")
print(f"Scaled XGBoost Training Time: {scale_xgb_train_time} seconds")


Linear Regression MSE: 6.0212570240006364
Linear Regression R^2: 0.6646905299320611
SVR MSE: 6.177533595623903
SVR R^2: 0.6559878596746443
XGBoost MSE: 5.910049667541645
XGBoost R^2: 0.6708834028842183
Scaled Linear Regression Training Time: 0.6109638214111328 seconds
Scaled SVR Training Time: 1.5580792427062988 seconds
Scaled XGBoost Training Time: 0.9362764358520508 seconds


In [327]:
"""OBSERVATION:
Scaling the data appears to reduce training times with GridCV noticeably.
MSEs appears to be unchanged however!
Also, SVR choose a linear kernel, rather than a polynomial one.
"""

'OBSERVATION:\nScaling the data appears to reduce training times with GridCV noticeably. \nMSEs appears to be unchanged however!\nAlso, SVR choose a linear kernel, rather than a polynomial one.\n'

##EXPERIMENT 2: Generate new features
• Polynomial features (2nd and 3rd order)


In [328]:
#2ND ORDER
#generate polynomial features derived from X data
from sklearn.preprocessing import PolynomialFeatures

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

#SCALE ONLY THE FEATURES!
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


#degree=2 for 2nd order features
poly_features = PolynomialFeatures(degree=3, include_bias=False)

#fit and transform the features
X_train = poly_features.fit_transform(X_train)
X_test = poly_features.transform(X_test)

#linear regression
lin_reg = LinearRegression()

t0 = time.time()
%memit lin_reg.fit(X_train, y_train)
t1 = time.time()
poly_lin_train_time = t1 - t0

t0 = time.time()
poly_y_pred_lin = lin_reg.predict(X_test)
t1 = time.time()
poly_lin_infer_time = t1 - t0

#SVR
param_grid = {'C': np.linspace(0.1, 1, 7), #higher C means less tolerance, lower means more tolerance
              'kernel': ['linear', 'rbf', 'poly'],
              'degree': [2, 3, 4,5],
              'epsilon':np.linspace(0.01, 0.1, 10)}

grid_search = GridSearchCV(SVR(), param_grid, cv=5, n_jobs=-1,verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
poly_svr_train_time = t1 - t0

t0 = time.time()
poly_y_pred_svr = grid_search.best_estimator_.predict(X_test)
t1 = time.time()
poly_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost

param_grid = {'n_estimators': [300],
              'n_jobs':[-1],
              'gamma': [1,1.5],
              'max_depth': [5,8],
              'eta': [0.1,0.3],
              'subsample':[1],
              'colsample_bytree':[0.75],
              'reg_lambda':[0.5,1],
              'reg_alpha':[3,2.5],
              }
#Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, n_jobs=-1, scoring = "neg_root_mean_squared_error", verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
poly_xgb_train_time = t1 - t0

t0 = time.time()
poly_y_pred_xgb = grid_search.best_estimator_.predict(X_test)
t1 = time.time()
poly_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)



peak memory: 605.98 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 840 candidates, totalling 4200 fits
peak memory: 606.23 MiB, increment: 0.00 MiB
Best SVR parameters: {'C': 1.0, 'degree': 2, 'epsilon': 0.1, 'kernel': 'linear'}
Fitting 5 folds for each of 32 candidates, totalling 160 fits
peak memory: 615.23 MiB, increment: 9.25 MiB
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.1, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 2.5, 'reg_lambda': 0.5, 'subsample': 1}


In [329]:
#evaluate
#lin reg
poly_lin_reg_mse = mean_squared_error(y_test, poly_y_pred_lin)
poly_lin_reg_r2 = r2_score(y_test, poly_y_pred_lin)

#svr
poly_svr_mse = mean_squared_error(y_test, poly_y_pred_svr)
poly_svr_r2 = r2_score(y_test, poly_y_pred_svr)

#xgb
poly_xgb_mse = mean_squared_error(y_test, poly_y_pred_xgb)
poly_xgb_r2 = r2_score(y_test, poly_y_pred_xgb)

print(f"Linear Regression MSE: {poly_lin_reg_mse}")
print(f"Linear Regression R^2: {poly_lin_reg_r2}")
print(f"SVR MSE: {poly_svr_mse}")
print(f"SVR R^2: {poly_svr_r2}")
print(f"XGBoost MSE: {poly_xgb_mse}")
print(f"XGBoost R^2: {poly_xgb_r2}")

print(f"Poly Linear Regression Training Time: {poly_lin_train_time} seconds")
print(f"Poly SVR Training Time: {poly_svr_train_time} seconds")
print(f"Poly XGBoost Training Time: {poly_xgb_train_time} seconds")


Linear Regression MSE: 16.242248176341487
Linear Regression R^2: 0.0955078637213751
SVR MSE: 5.305682591951444
SVR R^2: 0.7045391666283836
XGBoost MSE: 6.934081755189215
XGBoost R^2: 0.6138574936308825
Poly Linear Regression Training Time: 0.6842718124389648 seconds
Poly SVR Training Time: 1.828301191329956 seconds
Poly XGBoost Training Time: 2.4492132663726807 seconds


In [330]:
"""OBSERVATION:
Introducing a polynomial feature appears to significantly worsen the performance
of the linear regression model - this is expected since the purpose is to
introduce non-linearity to the data.

Polynomial features can introduce high multicollinearity (strong correlation among the features).
This can make it difficult for linear regression to estimate the
coefficients accurately and can lead to worse predictions.

SVR performed the best out of the three models - even with a linear kernel.
This may be because SVR does not assume a linear relationship between
X and Y and captures more complex relationships within the data
better than simple linear regression.

Adding 2nd order polynomial increased SVR and XGBoost performance,
but not linear regression - this indicates how XGBoost and SVR are better
at non-linear relationships.

However, increasing 2nd order to 3rd order made all models perform worse
than the last experiment.

2nd order results:
Linear Regression MSE: 15.811879945458458
Linear Regression R^2: 0.11947404600794098
SVR MSE: 4.148939564413031
SVR R^2: 0.7689554322059293
XGBoost MSE: 5.490413432665133
XGBoost R^2: 0.6942519458606913
Poly Linear Regression Training Time: 0.5376052856445312 seconds
Poly SVR Training Time: 5.408093452453613 seconds
Poly XGBoost Training Time: 12.322922945022583 seconds

3rd order results
Linear Regression MSE: 16.242248176341487
Linear Regression R^2: 0.0955078637213751
SVR MSE: 5.305682591951444
SVR R^2: 0.7045391666283836
XGBoost MSE: 6.934081755189215
XGBoost R^2: 0.6138574936308825
Poly Linear Regression Training Time: 0.5849087238311768 seconds
Poly SVR Training Time: 5.951260566711426 seconds
Poly XGBoost Training Time: 30.620221853256226 seconds
"""

'OBSERVATION:\nIntroducing a polynomial feature appears to significantly worsen the performance\nof the linear regression model - this is expected since the purpose is to\nintroduce non-linearity to the data. \n\nPolynomial features can introduce high multicollinearity (strong correlation among the features). \nThis can make it difficult for linear regression to estimate the \ncoefficients accurately and can lead to worse predictions.\n\nSVR performed the best out of the three models - even with a linear kernel. \nThis may be because SVR does not assume a linear relationship between \nX and Y and captures more complex relationships within the data\nbetter than simple linear regression.\n\nAdding 2nd order polynomial increased SVR and XGBoost performance, \nbut not linear regression - this indicates how XGBoost and SVR are better\nat non-linear relationships. \n\nHowever, increasing 2nd order to 3rd order made all models perform worse\nthan the last experiment. \n\n2nd order results:\nL

##EXPERIMENT 3: FEATURE TRANSFORMATION

In [331]:
#USE PCA
from sklearn.decomposition import PCA

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

#apply scaling
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#apply pca
pca = PCA(n_components=0.9)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

#lin reg
lin_reg = LinearRegression()

t0 = time.time()
%memit lin_reg.fit(X_train, y_train)
t1 = time.time()
pca_lin_train_time = t1 - t0

t0 = time.time()
pca_y_pred_lin = lin_reg.predict(X_test)
t1 = time.time()
pca_lin_infer_time = t1 - t0

#SVR
param_grid = {'C': np.linspace(0.1, 1, 7), #higher C means less tolerance, lower means more tolerance
              'kernel': ['linear', 'rbf', 'poly'],
              'degree': [2, 3, 4,5],
              'epsilon':np.linspace(0.01, 0.1, 10)}

grid_search = GridSearchCV(SVR(), param_grid, cv=5, n_jobs=-1, verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
pca_svr_train_time = t1 - t0

t0 = time.time()
pca_y_pred_svr = grid_search.best_estimator_.predict(X_test)
t1 = time.time()
pca_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost

param_grid = {'n_estimators': [300],
              'n_jobs':[-1],
              'gamma': [1,1.5],
              'max_depth': [5,8],
              'eta': [0.1,0.3],
              'subsample':[1],
              'colsample_bytree':[0.75],
              'reg_lambda':[0.5,1],
              'reg_alpha':[3,2.5],
              }
#Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, n_jobs=-1, scoring = "neg_root_mean_squared_error", verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
pca_xgb_train_time = t1 - t0

t0 = time.time()
pca_y_pred_xgb = grid_search.best_estimator_.predict(X_test)
t1 = time.time()
pca_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)



peak memory: 597.74 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 840 candidates, totalling 4200 fits
peak memory: 597.74 MiB, increment: 0.00 MiB
Best SVR parameters: {'C': 1.0, 'degree': 3, 'epsilon': 0.1, 'kernel': 'poly'}
Fitting 5 folds for each of 32 candidates, totalling 160 fits
peak memory: 597.74 MiB, increment: 0.00 MiB
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 1, 'subsample': 1}


In [332]:
#evaluate
#lin reg
pca_lin_reg_mse = mean_squared_error(y_test, pca_y_pred_lin)
pca_lin_reg_r2 = r2_score(y_test, pca_y_pred_lin)

#svr
pca_svr_mse = mean_squared_error(y_test, pca_y_pred_svr)
pca_svr_r2 = r2_score(y_test, pca_y_pred_svr)

#xgb
pca_xgb_mse = mean_squared_error(y_test, pca_y_pred_xgb)
pca_xgb_r2 = r2_score(y_test, pca_y_pred_xgb)

print(f"Linear Regression MSE: {pca_lin_reg_mse}")
print(f"Linear Regression R^2: {pca_lin_reg_r2}")
print(f"SVR MSE: {pca_svr_mse}")
print(f"SVR R^2: {pca_svr_r2}")
print(f"XGBoost MSE: {pca_xgb_mse}")
print(f"XGBoost R^2: {pca_xgb_r2}")

print(f"PCA Linear Regression Training Time: {pca_lin_train_time} seconds")
print(f"PCA SVR Training Time: {pca_svr_train_time} seconds")
print(f"PCA XGBoost Training Time: {pca_xgb_train_time} seconds")

print(f"PCA reduced to {pca.n_components_} components! ")

Linear Regression MSE: 5.839949811238412
Linear Regression R^2: 0.6747870970090805
SVR MSE: 5.935121810508435
SVR R^2: 0.6694871949266188
XGBoost MSE: 6.289780824268823
XGBoost R^2: 0.6497370787158678
PCA Linear Regression Training Time: 0.6138339042663574 seconds
PCA SVR Training Time: 1.6256346702575684 seconds
PCA XGBoost Training Time: 0.9283638000488281 seconds
PCA reduced to 6 components! 


In [333]:
"""OBSERVATION:
Reducing PCA to capture 90% of variance reduced the features to 6 components (out of 9).
However, this also appears to raise MSE for XGBoost, while lowering SVR and Linear regression.
Further experimentation with XGBoost parameters may be needed.
"""

'OBSERVATION:\nReducing PCA to capture 90% of variance reduced the features to 6 components (out of 9).\nHowever, this also appears to raise MSE for XGBoost, while lowering SVR and Linear regression.\nFurther experimentation with XGBoost parameters may be needed.\n'

##EXPERIMENT 4: HANDLING NOISE
Introduce synthetic noise:

• Add a random continuous feature (you can use random.Generator.uniform)

• Add a random discrete categorical feature

In [334]:
#Introduce synthetic noise:
#Add a random continuous feature using random.Generator.uniform


noise = np.random.rand(len(data))

#random discrete categorical feature, 2 classes
random_categories = np.random.randint(0, 2, size=len(data))
data['RANDOM_CAT'] = random_categories
data['NOISE'] = noise
#display(data)

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

#apply scaling
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#lin reg
lin_reg = LinearRegression()

t0 = time.time()
%memit lin_reg.fit(X_train, y_train)
t1 = time.time()
noise_lin_train_time = t1 - t0

t0 = time.time()
noise_y_pred_lin = lin_reg.predict(X_test)
t1 = time.time()
noise_lin_infer_time = t1 - t0

#SVR
param_grid = {'C': np.linspace(0.1, 1, 7), #higher C means less tolerance, lower means more tolerance
              'kernel': ['linear', 'rbf', 'poly'],
              'degree': [2, 3, 4,5],
              'epsilon':np.linspace(0.01, 0.1, 10)}

grid_search = GridSearchCV(SVR(), param_grid, cv=5, n_jobs=-1, verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
noise_svr_train_time = t1 - t0

t0 = time.time()
noise_y_pred_svr = grid_search.best_estimator_.predict(X_test)
t1 = time.time()
noise_svr_infer_time = t1 - t0

print(f"Best SVR parameters: %s" % grid_search.best_params_)

#XGBoost

param_grid = {'n_estimators': [300],
              'n_jobs':[-1],
              'gamma': [1,1.5],
              'max_depth': [5,8],
              'eta': [0.1,0.3],
              'subsample':[1],
              'colsample_bytree':[0.75],
              'reg_lambda':[0.5,1],
              'reg_alpha':[3,2.5],
              }
#Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 0.5, 'subsample': 1}

grid_search = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, n_jobs=-1, scoring = "neg_root_mean_squared_error", verbose=True)

t0 = time.time()
%memit grid_search.fit(X_train, y_train)
t1 = time.time()
noise_xgb_train_time = t1 - t0

t0 = time.time()
noise_y_pred_xgb = grid_search.best_estimator_.predict(X_test)
t1 = time.time()
noise_xgb_infer_time = t1 - t0

print(f"Best XGBoost parameters:%s" % grid_search.best_params_)



peak memory: 597.74 MiB, increment: 0.00 MiB
Fitting 5 folds for each of 840 candidates, totalling 4200 fits
peak memory: 597.74 MiB, increment: 0.00 MiB
Best SVR parameters: {'C': 1.0, 'degree': 2, 'epsilon': 0.01, 'kernel': 'linear'}
Fitting 5 folds for each of 32 candidates, totalling 160 fits
peak memory: 597.74 MiB, increment: 0.00 MiB
Best XGBoost parameters:{'colsample_bytree': 0.75, 'eta': 0.3, 'gamma': 1.5, 'max_depth': 5, 'n_estimators': 300, 'n_jobs': -1, 'reg_alpha': 3, 'reg_lambda': 1, 'subsample': 1}


In [335]:
#evaluate
#lin reg
noise_lin_reg_mse = mean_squared_error(y_test, noise_y_pred_lin)
noise_lin_reg_r2 = r2_score(y_test, noise_y_pred_lin)

#svr
noise_svr_mse = mean_squared_error(y_test, noise_y_pred_svr)
noise_svr_r2 = r2_score(y_test, noise_y_pred_svr)

#xgb
noise_xgb_mse = mean_squared_error(y_test, noise_y_pred_xgb)
noise_xgb_r2 = r2_score(y_test, noise_y_pred_xgb)

print(f"Linear Regression MSE: {noise_lin_reg_mse}")
print(f"Linear Regression R^2: {noise_lin_reg_r2}")
print(f"SVR MSE: {noise_svr_mse}")
print(f"SVR R^2: {noise_svr_r2}")
print(f"XGBoost MSE: {noise_xgb_mse}")
print(f"XGBoost R^2: {noise_xgb_r2}")

print(f"Noisy Linear Regression Training Time: {noise_lin_train_time} seconds")
print(f"Noisy SVR Training Time: {noise_svr_train_time} seconds")
print(f"Noisy XGBoost Training Time: {noise_xgb_train_time} seconds")


Linear Regression MSE: 5.770998488020319
Linear Regression R^2: 0.6786268320605133
SVR MSE: 7.273371059388836
SVR R^2: 0.5949632799579863
XGBoost MSE: 6.547074497418209
XGBoost R^2: 0.6354090065456097
Noisy Linear Regression Training Time: 0.662168025970459 seconds
Noisy SVR Training Time: 1.5605535507202148 seconds
Noisy XGBoost Training Time: 0.9128923416137695 seconds


##Model Efficiency

In [337]:
#TIME COMPARISONS
time_df = pd.DataFrame(columns=['Model Variant', 'Training Time', 'Inference Time'])

#Linear Regression
time_df.loc[len(time_df)] = ['BASE Linear Regression', base_lin_train_time, base_lin_infer_time]
time_df.loc[len(time_df)] = ['SCALED Linear Regression', scale_lin_train_time, scale_lin_infer_time]
time_df.loc[len(time_df)] = ['POLY Linear Regression', poly_lin_train_time, poly_lin_infer_time]
time_df.loc[len(time_df)] = ['PCA Linear Regression', pca_lin_train_time, pca_lin_infer_time]
time_df.loc[len(time_df)] = ['NOISY Linear Regression', noise_lin_train_time, noise_lin_infer_time]

#SVR
time_df.loc[len(time_df)] = ['BASE SVR', base_svr_train_time, base_svr_infer_time]
time_df.loc[len(time_df)] = ['SCALED SVR', scale_svr_train_time, scale_svr_infer_time]
time_df.loc[len(time_df)] = ['POLY SVR', poly_svr_train_time, poly_svr_infer_time]
time_df.loc[len(time_df)] = ['PCA SVR', pca_svr_train_time, pca_svr_infer_time]
time_df.loc[len(time_df)] = ['NOISY SVR', noise_svr_train_time, noise_svr_infer_time]

#XGBoost
time_df.loc[len(time_df)] = ['BASE XGBoost', base_xgb_train_time, base_xgb_infer_time]
time_df.loc[len(time_df)] = ['SCALED XGBoost', scale_xgb_train_time, scale_xgb_infer_time]
time_df.loc[len(time_df)] = ['POLY XGBoost', poly_xgb_train_time, poly_xgb_infer_time]
time_df.loc[len(time_df)] = ['PCA XGBoost', pca_xgb_train_time, pca_xgb_infer_time]
time_df.loc[len(time_df)] = ['NOISY XGBoost', noise_xgb_train_time, noise_xgb_infer_time]

display(time_df)

"""
If required to deploy a model, which trade-offs are necessary?

If speed is important: Linear regression offers the fastest speeds, and decent
accuracy for this dataset (except for when adding Polynomial features!).

If accuracy is important: SVR outperformed linear and XGBoost in some cases,
although linear had good accuracy as well, along with its speed.
XGBoost requires much more hyperparameter experimentation to be considered as the best.

If interpretability is important: Linear Regression is the preferred choice,
as it offers a clearer understanding of feature importance and relationships.
"""


,Model Variant,Training Time,Inference Time
0,BASE Linear Regression,0.673842,0.000947
1,SCALED Linear Regression,0.610964,0.000310
2,POLY Linear Regression,0.684272,0.005239
3,PCA Linear Regression,0.613834,0.000323
4,NOISY Linear Regression,0.662168,0.000340
5,BASE SVR,2.388616,0.000661
6,SCALED SVR,1.558079,0.000751
7,POLY SVR,1.828301,0.000646
8,PCA SVR,1.625635,0.000801
9,NOISY SVR,1.560554,0.000773


'\nIf required to deploy a model, which trade-offs are necessary?\n\nIf inference speed is important: Consider deploying Linear Regression or XGBoost with scaled hyperparameters for faster predictions.\nIf accuracy is important: Choose SVR or XGBoost with a focus on maximizing performance, even if it leads to slower inference or larger model size.\nIf interpretability is important: Linear Regression might be the preferred choice, as it offers a clear understanding of feature importance and relationships.\n'

##Summarization of findings


In [349]:
#SUMMARIZATION TABLE
mse_df = pd.DataFrame(columns=['Model Variant', 'MSE', 'R-squared'])
#linear reg errors
mse_df.loc[len(mse_df)] = ['BASE Linear Regression MSE', lin_reg_mse, lin_reg_r2]
mse_df.loc[len(mse_df)] = ['SCALED Linear Regression MSE', scale_lin_reg_mse, scale_lin_reg_r2]
mse_df.loc[len(mse_df)] = ['POLY Linear Regression MSE', poly_lin_reg_mse, poly_lin_reg_r2]
mse_df.loc[len(mse_df)] = ['PCA Linear Regression MSE', pca_lin_reg_mse, pca_lin_reg_r2]
mse_df.loc[len(mse_df)] = ['NOISY Linear Regression MSE', noise_lin_reg_mse, noise_lin_reg_r2]

#SVR
mse_df.loc[len(mse_df)] = ['BASE SVR MSE', svr_mse, svr_r2]
mse_df.loc[len(mse_df)] = ['SCALED SVR MSE', scale_svr_mse, scale_svr_r2]
mse_df.loc[len(mse_df)] = ['POLY SVR MSE', poly_svr_mse, poly_svr_r2]
mse_df.loc[len(mse_df)] = ['PCA SVR MSE', pca_svr_mse, pca_svr_r2]
mse_df.loc[len(mse_df)] = ['NOISY SVR MSE', noise_svr_mse, noise_svr_r2]

#XGBoost
mse_df.loc[len(mse_df)] = ['BASE XGBoost MSE', xgb_mse, xgb_r2]
mse_df.loc[len(mse_df)] = ['SCALED XGBoost MSE', scale_xgb_mse, scale_xgb_r2]
mse_df.loc[len(mse_df)] = ['POLY XGBoost MSE', poly_xgb_mse, poly_xgb_r2]
mse_df.loc[len(mse_df)] = ['PCA XGBoost MSE', pca_xgb_mse, pca_xgb_r2]
mse_df.loc[len(mse_df)] = ['NOISY XGBoost MSE', noise_xgb_mse, noise_xgb_r2]

display(mse_df)

print(f"Lowest MSE model variant: {mse_df.loc[mse_df['MSE'].idxmin()]['MSE']} {mse_df.loc[mse_df['MSE'].idxmin()]['Model Variant']} ")
print(f"Highest MSE model variant: {mse_df.loc[mse_df['MSE'].idxmax()]['MSE']} {mse_df.loc[mse_df['MSE'].idxmax()]['Model Variant']} ")

,Model Variant,MSE,R-squared
0,BASE Linear Regression MSE,6.021257,0.664691
1,SCALED Linear Regression MSE,6.021257,0.664691
2,POLY Linear Regression MSE,16.242248,0.095508
3,PCA Linear Regression MSE,5.839950,0.674787
4,NOISY Linear Regression MSE,5.770998,0.678627
5,BASE SVR MSE,6.761788,0.623452
6,SCALED SVR MSE,6.177534,0.655988
7,POLY SVR MSE,5.305683,0.704539
8,PCA SVR MSE,5.935122,0.669487
9,NOISY SVR MSE,7.273371,0.594963


Lowest MSE model variant: 5.305682591951444 POLY SVR MSE 
Highest MSE model variant: 16.242248176341487 POLY Linear Regression MSE 


##Final model choice

it appears that SVR with added polynomial features (specifically, 2nd order) achieved the lowest MSE and highest R-squared compared to other model variants - therefore, this is my recomended model for the given dataset. This suggests that this model effectively captured the non-linear relationships introduced in the dataset, leading to improved predictive performance.

The inclusion of polynomial features introduced non-linearity into the model, allowing it to better represent complex patterns in the data that the model previously s struggled with.

##Proposed Further Refinements
If more time, and patience, were available, a more extensive GridSearch over a wider range of values could potentially yield better results. Using techniques like RandomizedSearchCV or Bayesian Optimization would alternatively allow for a more efficient exploration of the hyperparameter space.

Experimenting with other models, particularly  Ensemble Methods could potentially improve overall accuracy and stability.

